In [1]:
import requests
import json
import plotly.express as px

import numpy as np

from datetime import datetime
import pandas as pd

# custom libraries
from data import Data
from util import *
from visualization import *

In [2]:
# Leemos api.json
with open('api.json') as file:
    api = json.load(file)
    clientID = api['IGDB']['clientID']
    clientSecret = api['IGDB']['clientSecret']

In [3]:
# Creamos objeto Data
data = Data(clientID = clientID, clientSecret = clientSecret)

In [4]:
# Extraemos datos de IGDB utilizando su API
data.extract(endpoints = ['games', 'game_engines', 'language_supports', 'languages', 'genres'], batches = 10000, show_logs = True, keep_logs = True, save_csv = True)

[REQUEST] | 07/31/23 14:22:57 | Status 200 | Endpoint: https://api.igdb.com/v4/games | Batch 1 | Response time [1.18s] | Time elapsed [1.18s]
[REQUEST] | 07/31/23 14:22:58 | Status 200 | Endpoint: https://api.igdb.com/v4/games | Batch 2 | Response time [0.81s] | Time elapsed [2.0s]
[REQUEST] | 07/31/23 14:22:59 | Status 200 | Endpoint: https://api.igdb.com/v4/games | Batch 3 | Response time [1.12s] | Time elapsed [3.15s]
[REQUEST] | 07/31/23 14:22:59 | Status 200 | Endpoint: https://api.igdb.com/v4/games | Batch 4 | Response time [0.5s] | Time elapsed [3.74s]
[REQUEST] | 07/31/23 14:23:01 | Status 200 | Endpoint: https://api.igdb.com/v4/games | Batch 5 | Response time [1.08s] | Time elapsed [4.82s]
[REQUEST] | 07/31/23 14:23:02 | Status 200 | Endpoint: https://api.igdb.com/v4/games | Batch 6 | Response time [1.14s] | Time elapsed [5.97s]
[REQUEST] | 07/31/23 14:23:03 | Status 200 | Endpoint: https://api.igdb.com/v4/games | Batch 7 | Response time [1.14s] | Time elapsed [7.12s]
[REQUEST

In [4]:
# Leemos los datos extraidos
data.read_csvs(paths = ['games_data.csv', 'game_engines_data.csv', 'language_supports_data.csv', 'languages_data.csv', 'genres_data.csv'])

/home/cobayaculera/bootcamp/PROJECTS/Project 1 ETL/data.py:50: DtypeWarning: Columns (57) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


In [5]:
# Nos quedamos solamente con las columnas que nos interesan
data.filterColumns(columns = ['id', 'name', 'language_supports', 'game_engines', 'first_release_date', 'genres', 'category'], inplace = True)

In [6]:
# Columnas con las que se va a trabajar
data.main

,id,name,language_supports,game_engines,first_release_date,genres,category
0,35642,Dungeon Crawlers HD,"[76103, 76104, 76105]",NaN,1.433117e+09,"[12, 15, 16, 24, 32]",0
1,246925,Stickman and the Sword of Legends,"[678670, 678671, 678672]",NaN,1.682640e+09,"[31, 32]",0
2,245087,Cry Baby,NaN,NaN,6.836832e+08,[8],0
3,85450,Transformers Prime: The Game,NaN,NaN,NaN,NaN,0
4,95080,Dotra,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...
241299,2598,Teenage Mutant Ninja Turtles: Out of the Shadows,"[87425, 87426, 87427, 87428, 87429, 87430]",[6],1.377648e+09,"[25, 31]",0
241300,144598,Moorhuhn VR,NaN,NaN,1.514678e+09,[5],0
241301,121767,The Big Con,"[288072, 288073, 288074, 288075, 288076, 28807...",NaN,1.630368e+09,"[2, 31, 32]",0
241302,248598,Attack of the Silver Ball,NaN,NaN,1.037578e+09,"[30, 33]",0


In [7]:
# Se crean columnas en el dataframe de language_supports, una por cada tipo de language_support_type
# Excluimos el language_support para la interfaz porque no nos parece un dato interesante
data.splitColumn(data_frame = 'language_supports',
                column = 'language',
                query_field = 'language_support_type',
                queries = [1,2],
                inplace = True
                 )

# Se crean dos columnas nuevas con supports para audio y subtitulos con la copia de la columna de language_supports
data.main['audio_language_supports'] = data.main['language_supports']
data.main['subtitles_language_supports'] = data.main['language_supports']

# Se modifican los id's de la tabla language_support por los id's de la tabla languages
data.parseLists(columns = ['audio_language_supports', 'subtitles_language_supports'],
                data_frames = ['language_supports', 'language_supports'],
                fields = ['language_1', 'language_2'],
                inplace = True
               )

# Se eliminan todos los np.nan que se encuentran dentro de listas
# Si la lista está compuesta completamente por np.nan, se sustituye por un np.nan
data.main['audio_language_supports'] = data.main['audio_language_supports'].apply(removeNaFromLists).apply(removeEmptyLists)
data.main['subtitles_language_supports'] = data.main['subtitles_language_supports'].apply(removeNaFromLists).apply(removeEmptyLists)

# Se cambian los id's de la tabla languages por los nombres de los lenguajes
data.parseLists(columns = ['audio_language_supports', 'subtitles_language_supports'],
                data_frames = ['languages', 'languages'],
                fields = ['name', 'name'],
                inplace = True
               )

In [8]:
# Convertimos timestamp en un objeto datetime
data.main['first_release_date'] = data.main['first_release_date'].apply(lambda x : datetime.fromtimestamp(x).strftime('%m-%d-%Y') if not pd.isna(x) else np.nan)

# Creamos columna con los meses
data.main['month_release'] = data.main['first_release_date'].apply(lambda x : x[:2] if not pd.isna(x) else np.nan)

# Creamos columna con los años
data.main['year_release'] = data.main['first_release_date'].apply(lambda x : x[-4:] if not pd.isna(x) else np.nan)

In [9]:
# Se cambian los id's de la tabla game_engines por los nombres de los motores
data.parseLists(columns = ['game_engines'],
                data_frames = ['game_engines'],
                fields = ['name'],
                inplace = True
               )

In [10]:
# Se cambian los id's de la tabla genres por los nombres de los géneros
data.parseLists(columns = ['genres'],
                data_frames = ['genres'],
                fields = ['name'],
                inplace = True
               )

In [11]:
# Se cambian los id's de las categorías por los nombres
categorys = diccionario = {0: 'main_game',
                           1: 'dlc_addon',
                           2: 'expansion',
                           3: 'bundle',
                           4: 'standalone_expansion',
                           5: 'mod',
                           6: 'episode',
                           7: 'season',
                           8: 'remake',
                           9: 'remaster',
                           10: 'expanded_game',
                           11: 'port',
                           12: 'fork',
                           13: 'pack',
                           14: 'update'}


data.main['category'] = data.main['category'].replace(categorys)

In [12]:
# Nos quedamos con las columnas procesadas
data.filterColumns(columns = ['id',
                              'name',
                              'game_engines',
                              'genres',
                              'category',
                              'audio_language_supports',
                              'subtitles_language_supports',
                              'month_release',
                              'year_release'
                             ],
                   inplace = True
                  )

In [13]:
# Datos procesados

data.main

,id,name,game_engines,genres,category,audio_language_supports,subtitles_language_supports,month_release,year_release
0,35642,Dungeon Crawlers HD,NaN,"[Role-playing (RPG), Strategy, Turn-based stra...",main_game,[English],[English],05,2015
1,246925,Stickman and the Sword of Legends,NaN,"[Adventure, Indie]",main_game,[English],[English],04,2023
2,245087,Cry Baby,NaN,[Platform],main_game,NaN,NaN,08,1991
3,85450,Transformers Prime: The Game,NaN,NaN,main_game,NaN,NaN,NaN,NaN
4,95080,Dotra,NaN,NaN,main_game,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
241299,2598,Teenage Mutant Ninja Turtles: Out of the Shadows,[Unreal Engine],"[Hack and slash/Beat 'em up, Adventure]",main_game,[English],NaN,08,2013
241300,144598,Moorhuhn VR,NaN,[Shooter],main_game,NaN,NaN,12,2017
241301,121767,The Big Con,NaN,"[Point-and-click, Adventure, Indie]",main_game,[English],"[English, French, German, Spanish (Spain), Por...",08,2021
241302,248598,Attack of the Silver Ball,NaN,"[Pinball, Arcade]",main_game,NaN,NaN,11,2002


In [15]:
key = api['Airtable']['key']
app = api['Airtable']['app']
tbls = api['Airtable']['tbls']

In [16]:
loadToAirtable(key, app, tbls, data.main) #tarda aprox 4 horas y media

In [17]:
dataframe = extractFromAirtable(key, app, tbls) #tarda menos de media hora

In [15]:
languages_fig(dataframe)

In [16]:
releases_fig(dataframe)

In [17]:
engines_fig(dataframe)

In [18]:
engines_years_fig(dataframe)

In [19]:
genres_categories_fig(dataframe)

In [ ]:
###############################################################################################################################################################################